<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Procesamiento de Lenguaje Natural</h1>

<h3>RNN a pie</h3>


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/pln/blob/main/labs/RNN/Estados-ocultos.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

<p>
Tomado parcialmente y adaptado de libretas de la <i>Especialización en procesamiento de lenguaje natural</i> de <i>Deeplearning.ai</i>, disponible en <i>Coursera</i>.
</p>


</center>


## RNN vainilla

Para una RNN simple como se muestra en la figura:

![vanilla rnn](https://github.com/mcd-unison/pln/blob/main/labs/RNN/vanilla_rnn.PNG?raw=1)


La activación de los estados ocultos están dados por:      

$h^{<t>}=g(W_{hh}h^{<t-1>} + W_{hx}x^{<t>} + b_h)$                                        


este ejemplo lo vams a hacer usando exclusivamente `numpy` para entender el modelo.

In [ ]:
import numpy as np
from time import perf_counter

Vamos entonces a desarrollar la función de alimentación a adelante de una RNN

In [ ]:
def sigmoid(x):
    # Calcula la función logística

    ## INICIA CODIGO
    return 1 / (1 + np.exp(-x))
    ## ACABA CODIGO

In [ ]:
def forward_V_RNN(inputs, weights):
    # Forward propagation para una RNN vanilla
    x_t, h_t_prev = inputs

    # weights.
    w_hh, w_xh, b_h = weights

    ### INICIA CODIGO ###
    # Nuevo estado oculto

    # Operación lineal
    z_t = np.dot(w_xh, x_t) + np.dot(w_hh, h_t_prev) + b_h

    # Activación
    h_t = np.tanh(z_t)

    ### ACABA CODIGO ###

    return h_t

Vamos a probar como funciona

In [ ]:
# Data

nh = 2   # Dimensión del vector de variables ocultas
nx = 3   # Dimensión del vector de entrada

w_hh = np.full((nh, nh), 1.)  # 3x2 llenado con puros 1s
w_hx = np.full((nh, nx), 9.)  # 3x3 llenado con puros 9s
h_t_prev = np.full((nh, 1), 1.)  # 2x1 llenado con puros 1s
x_t = np.full((nx, 1), 9.)       # 3x1 llenado con puros 9s
b_h = np.zeros((nh, 1))

# Si prefieres valores aleatorios, descomenta lo siguiente:

# w_hh = np.random.standard_normal((nh,nh))
# w_hx = np.random.standard_normal((nh,nx))
# h_t_prev = np.random.standard_normal((nh,1))
# x_t = np.random.standard_normal((nx,1))

# Aplicando un solo paso
h_t = forward_V_RNN([x_t, h_t_prev], [w_hh, w_hx, b_h])

print("\nValor h_t:")
print(h_t, "\n")




Valor h_t:
[[1.]
 [1.]] 



## RNN tipo LSTM

Una LST es un modelo como el que se muestra en la figura, con todo y sus ecuaciones

![](https://github.com/mcd-unison/pln/blob/main/labs/RNN/LSTM.jpg?raw=1)

Como podemos ver tenemos 3 vectores de entrada a la celda:

- $h^{<t-1>}$ el vector de variables ocultas provenientes de un paso anterior,
- $C^{<t-1>}$ el vector de valores de celda (memoria de largo plazo) provenientes de un paso anterior,
- $x^{<t>}$ el vector de variables de entrada. Idealmente debería estar normalizado entre -1 y 1 cada uno de los valores de entrada.

Como podemos ver tenemos varias operaciones:

- Una compuerta de olvido $f$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda anterior (memoria de largo plazo)

- Una compuerta de entrada $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener la activación de la celda actual (memoria de corto plazo)

- Una compuerta de salida $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda actual en el valor de la de la variable oculta correspondiente.

- El calculo de la activación actual, que depende de $h^{<t-1>}$ y $x^{<t>}$, el cual se hace con una tangente hiperbólica, para mantener los valores entre -1 y 1.


Hagamos entonces una celda LSTM


In [ ]:
def forward_LSTM(inputs, weights):
    # Forward propagation para una RNN tipo LSTM
    x_t, h_t_prev, C_t_prev = inputs

    # pesos.
    Ui, Wi, Uf, Wf, Uo, Wo, U, W = weights

    ### INICIA CODIGO ###
    # Compuerta de entrada
    i = sigmoid(np.dot(Ui, x_t) + np.dot(Wi, h_t_prev))

    # Compuerta de olvido
    f = sigmoid(np.dot(Uf, x_t) + np.dot(Wf, h_t_prev))

    # Compuerta de salida
    o = sigmoid(np.dot(Uo, x_t) + np.dot(Wo, h_t_prev))

    # Valor de celda de memoria de corto plazo
    C_t_short = np.tanh(np.dot(U, x_t) + np.dot(W, h_t_prev))

    # Valor de celda de memoria de corto y largo plazo
    C_t = f * C_t_prev + i * C_t_short

    # Valor de variable oculta
    h_t = o * np.tanh(C_t)
    ### ACABA CODIGO ###

    return h_t, C_t

Vamos a probar como funciona

In [ ]:
# Data

nh = 2   # Dimensión del vector de variables ocultas
nx = 3   # Dimensión del vector de entrada

Ui = np.random.standard_normal((nh,nx))
Wi = np.random.standard_normal((nh,nh))

Uf = np.random.standard_normal((nh,nx))
Wf = np.random.standard_normal((nh,nh))

Uo = np.random.standard_normal((nh,nx))
Wo = np.random.standard_normal((nh,nh))

U = np.random.standard_normal((nh,nx))
W = np.random.standard_normal((nh,nh))


h_t_prev = 2 * np.random.standard_normal((nh,1)) - 1
C_t_prev = np.random.standard_normal((nh,1))
x_t = 2 * np.random.standard_normal((nx,1)) - 1

# Aplicando un solo paso
h_t, C_t = forward_LSTM(
    [x_t, h_t_prev, C_t_prev],
    [Ui, Wi, Uf, Wf, Uo, Wo, U, W]
)

print("\nValor h_t:")
print(h_t, "\n")

print("\nValor C_t:")
print(C_t, "\n")



Valor h_t:
[[ 0.16974741]
 [-0.67171261]] 


Valor C_t:
[[ 0.2026316 ]
 [-0.87955995]] 



## La función `scan`para el cálculo de BPTT

La función `scan` se usa para calcular la propagación hacia adelante. Si la funcións e implementa en un *framework* como *Tensorflow* o *pytorch*, entonces se puede ir guardando los gradientes de cada aplicación a lo largo del tiempo y usarlos en el calculo del gradiente para la función de aprendizaje.

Aquí solo vamos a tratar de mostrar como funcionaría dicha función, la cual recibe:

- `elems` : lista de entradas (`X`)
- `weights` : los parámetros que necesita la función de feedforward para su cálculo (pesos)
- `h_0` : estado oculto inicial

`scan` va por todos los valores de `x` en `elems`, llama la función de feedforward con los argumentos necesarios, guarda el estado oculto `h_t` y agrega el valor de `h_t` a una lista.

Vamos a hacer la función de scan para una celda tipo RNN vainilla

In [ ]:
def scan_V_RNN(elems, weights, h_0=None): # Forward propagation for RNNs
    h_t = h_0
    h = []
    for x in elems:
        h_t = forward_V_RNN([x, h_t], weights)
        h.append(h_t)
    return h, h_t

Vamos a probar inicializando una posible red RNN vainilla en un probable pornblema de PLN

In [ ]:
np.random.seed(10)

emb = 128                       # Embedding
T = 256                         # Tamaño de secuencia de tokens
h_dim = 16                      # Estados ocultos

h_0 = np.zeros((h_dim, 1))      # Estado inicial

# Inicialización aleatoria de pesos y sesgos
Whh = np.random.standard_normal((h_dim, h_dim))
Wxh = np.random.standard_normal((h_dim, emb))
bh = np.random.standard_normal((h_dim, 1))

# Inicialización aleatoria de una secuencia de tokens (en embeddings)
X = np.random.standard_normal((T, emb, 1))

weights = [Whh, Wxh, bh]

In [ ]:
# vanilla RNNs
tic = perf_counter()
h, h_T = scan_V_RNN(X, weights, h_0)
toc = perf_counter()
RNN_time=(toc-tic)*1000
print (f"Tomó {RNN_time:.2f}ms ejecutar el método de RNN vainilla.")


Tomó 4.36ms ejecutar el método de RNN vainilla.


**Desarrolla la función de scan para LSTM y prueba con la misma secuencia de entradas para una red LSTM**

In [ ]:
# Función scan para LSTM

# INICIA CODIGO
def scan_LSTM(elems, weights, h_0=None, C_0=None): # Forward propagation for LSTMs
    h_t = h_0
    C_t = C_0
    h = []
    for x in elems:
        h_t, C_t = forward_LSTM([x, h_t, C_t], weights)
        h.append(h_t)
    return h, (h_t, C_t)
# TERMINA CODIGO

In [ ]:
# Inicialización de variables

# INICIA CODIGO
def init_weights_LSTM(input_size, hidden_size):
    Ui = np.random.randn(hidden_size, input_size)
    Wi = np.random.randn(hidden_size, hidden_size)
    Uf = np.random.randn(hidden_size, input_size)
    Wf = np.random.randn(hidden_size, hidden_size)
    Uo = np.random.randn(hidden_size, input_size)
    Wo = np.random.randn(hidden_size, hidden_size)
    U = np.random.randn(hidden_size, input_size)
    W = np.random.randn(hidden_size, hidden_size)

    return Ui, Wi, Uf, Wf, Uo, Wo, U, W

# Tamaño de la entrada y oculta
input_size = 10
hidden_size = 20

# Inicialización de los pesos
weights_lstm = init_weights_LSTM(input_size, hidden_size)

# Inicialización de las variables ocultas y de celda para la primera iteración
h_0 = np.zeros((hidden_size, 1))
C_0 = np.zeros((hidden_size, 1))
# TERMINA CODIGO

In [ ]:
# Probando la función de scan

# INICIA CODIGO
# Definir una secuencia de entrada
input_sequence = [np.random.randn(input_size, 1) for _ in range(5)]

# Utilizar la función scan_LSTM para propagación hacia adelante en la secuencia de entrada
output_sequence, (last_hidden_state, last_cell_state) = scan_LSTM(input_sequence, weights_lstm, h_0, C_0)

# Imprimir el resultado
print("Secuencia de entrada:")
for i, x in enumerate(input_sequence):
    print(f"Entrada {i+1}: {x.flatten()}")

print("\nSecuencia de salida (estados ocultos):")
for i, h in enumerate(output_sequence):
    print(f"Estado oculto {i+1}: {h.flatten()}")

print("\nÚltimo estado oculto:")
print(last_hidden_state.flatten())

print("\nÚltimo estado de celda:")
print(last_cell_state.flatten())
# TERMINA CODIGO

Secuencia de entrada:
Entrada 1: [-1.13921105  0.28355966 -0.86445759 -0.56315377  0.9032777  -1.14493097
  0.7405767   0.53314369 -0.60216006  0.73046825]
Entrada 2: [-0.34189771 -0.06076702 -0.40615267 -0.50510426 -0.32214639 -0.01813614
  2.12555923  0.88685488  0.59836685 -0.3018719 ]
Entrada 3: [ 0.87248602  0.05104538  1.18634828  0.13499416 -1.03039271  0.21557975
  0.06723213 -2.21552296  0.95275411  1.96421665]
Entrada 4: [-1.54913261  1.16203116  1.88062467 -0.48011658  2.22176779  0.88767564
 -0.80055666 -0.08971248  0.0086192  -0.44158074]
Entrada 5: [ 0.46253816 -1.28167229  0.06349504  0.76509024 -1.32640181 -2.13303336
  0.74792873  3.59534737  0.4653044  -0.96877907]

Secuencia de salida (estados ocultos):
Estado oculto 1: [ 0.39528538  0.12754263  0.44041048 -0.02151237 -0.00148225  0.32393762
  0.18997142  0.09597162 -0.00790158  0.18548252 -0.00409817  0.02108608
 -0.01184834  0.04886019  0.0092828  -0.44960573  0.04164985 -0.12919259
  0.15776953 -0.15342377]
Estado